# Library


In [1]:
import pandas as pd
import os
import pyexasol

# folder

In [2]:
raw1_fold = 'D:/data/big2/BD1/raw'
df1_fold = 'D:/data/big2/BD1/df'
raw3_fold = 'D:/data/big2/BD3/raw'
df3_fold = 'D:/data/big2/BD3/df'

In [3]:
sys_fold = 'D:/data/big2/sys'

# server

In [4]:
# # insider db
# wd = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
# we = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSE')

In [5]:
# exasol db
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSE')
# wbt = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='wbt')

# 분석 테이블 리스트

In [104]:
# 분석 결과 테이블 41개
# 분석 참고 테이블 3개
# 동양 요청 테이블 2개
varchar_type_list = [
    # 'STD_BD_KOSIS', # 분석시 참고 테이블
    'STD_BD_GRD4_CAR_CURSTT', # 과제1
    'STD_BD_GRD4_CAR_CURSTT_TOT', # 과제1
    'STD_BD_GRD4_ELPDSRC_CURSTT', # 과제1
    'STD_BD_GRD4_MLSFC_RSLT', # 과제1
    'STD_BD_DAT_GRD4_MLSFC', # 과제1
    'STD_BD_DAT_GRD4_DTL_INFO', # 과제1
    'STD_BD_GRD4_RGN_CURSTT', # 과제1
    'STD_BD_GRD4_RGN_CURSTT_MOD', # 과제1
    'STD_BD_DAT_GRD4_CAR_CURSTT', # 과제1
    'STD_BD_GRD4_EXHST_GAS_MSS_TOT', # 과제1
    'STD_BD_GRD4_EXHST_GAS_MSS', # 과제1
    'STD_BD_DAT_GRD4_MEVLU', # 과제1
    'STD_BD_CAR_CURSTT_MOD', # 과제3
    'STD_BD_CAR_CURSTT_MOD2', # 과제3
    'STD_BD_CAR_CURSTT', # 과제3
    'STD_BD_CAR_REG_MKCNT', # 과제3
    'STD_BD_CAR_PRET', # 과제3
    'STD_BD_HYBRD_CAR_PRET', # 과제3
    'STD_BD_FUEL_GRD_VHCL_CURSTT_PRET', # 과제3
    'STD_BD_ECO_CAR_PRET', # 과제3
    'STD_BD_ERSR_RSLT', # 과제3
    'STD_BD_DAT_GRD_CURSTT', # 과제3
    'STD_BD_DAT_FUEL_CAR_DEC', # 과제3
    'STD_BD_SEASON_CRDN_NOCS_CURSTT', # 과제3
    'STD_BD_ORDITM_DSCL_CURSTT', # 과제3
    'STD_BD_SEASON_DSCL_RGN_CURSTT', # 과제3
    'STD_BD_DAT_RUN_LMT_CURSTT', # 과제3
    'STD_BD_GRD4_DS_CRRLTN_CFFCNT', # 과제3
    'STD_BD_GRD4_LEM_PRIO_ORD_SELCT_CURSTT', # 과제3
    'STD_BD_GRD5_DS_CRRLTN_CFFCNT', # 과제3
    'STD_BD_GRD5_LEM_PRIO_ORD_SELCT_CURSTT', # 과제3
    'STD_BD_DAT_LEM_SELCT', # 과제3
    'STD_BD_DAT_SELCT_PNT_CURSTT', # 과제3
]
normal_type_list = [
    # 'STD_BD_TB_MAPDATA', # 분석시 참고 테이블
    # 'STD_BD_GRD4_RESULT', # 분석시 참고 테이블
    'STD_BD_DAT_GRD4_CERT_NO_RVW', # 과제1
    'STD_BD_DAT_GRD4_SI', # 과제1
    'STD_BD_GRD4_SI', # 과제1
    'STD_BD_DAT_GRD4_EXHST_MSS_CURSTT', # 과제1
    'STD_BD_GRD5_ELPDSRC_CURSTT', # 과제3
    'STD_BD_GRD5_LEM_N_MOD', # 과제3
    'STD_BD_SEASON_DY_AVRG_CRDN_NOCS', # 과제3
    'STD_BD_DAT_GRD5_REDUC_BIZ', # 과제3
    # 'STD_BD_CAR_CURSTT_MOD_4RD_INS', # 동양 요청 테이블
    # 'STD_BD_CAR_CURSTT_MOD2_4RD_INS', # 동양 요청 테이블
]
len(varchar_type_list), len(normal_type_list)

(33, 8)

# Load

## STD_BD_TB_MAPDATA(RH에서 제공한 법정동코드)

In [5]:
# # name = 'tb_mapdata_gu(2023.04.20)'
# name = 'STD_BD_TB_MAPDATA'
# file_name = f'{name}.csv'
# STD_BD_TB_MAPDATA = pd.read_csv(os.path.join(sys_fold, file_name), low_memory=False)
# STD_BD_TB_MAPDATA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3799 entries, 0 to 3798
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DONG_CODE    3799 non-null   int64  
 1   CTPRVN_CODE  3798 non-null   float64
 2   SIGNGU_CODE  3798 non-null   float64
 3   SDSTRD_CODE  3798 non-null   float64
 4   CTPRVN_NM    3799 non-null   object 
 5   SIGNGU_NM    3757 non-null   object 
 6   SDSTRD_NM    3521 non-null   object 
dtypes: float64(3), int64(1), object(3)
memory usage: 207.9+ KB


In [6]:
# expdf = STD_BD_TB_MAPDATA
# table_nm = 'STD_BD_TB_MAPDATA'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 1s
# we.import_from_pandas(expdf, table_nm)

In [7]:
# sql

'create or replace table STD_BD_TB_MAPDATA( \nDONG_CODE number,\nCTPRVN_CODE float,\nSIGNGU_CODE float,\nSDSTRD_CODE float,\nCTPRVN_NM varchar(255),\nSIGNGU_NM varchar(255),\nSDSTRD_NM varchar(255)\n)'

## STD_BD_GRD4_RESULT(4등급 result)

In [8]:
# # 9.9s
# # name = '4등급_차량_리스트_양식_result_최종(전체)(2023.07.03)'
# name = 'STD_BD_GRD4_RESULT'
# file_name = f'{name}.csv'
# STD_BD_GRD4_RESULT = pd.read_csv(os.path.join(sys_fold, file_name), low_memory=False)
# STD_BD_GRD4_RESULT.info()

In [9]:
# expdf = STD_BD_GRD4_RESULT
# table_nm = 'STD_BD_GRD4_RESULT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 1s
# we.import_from_pandas(expdf, table_nm)

In [10]:
# sql

## STD_BD_KOSIS

In [11]:
# # 9.9s
# name = 'STD_BD_KOSIS'
# file_name = f'{name}.csv'
# STD_BD_KOSIS = pd.read_csv(os.path.join(sys_fold, file_name), low_memory=False)
# STD_BD_KOSIS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1230 entries, 0 to 1229
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CTPV                 1230 non-null   object 
 1   SGG                  1230 non-null   object 
 2   VHCTY_CD             1230 non-null   object 
 3   DY_AVRG_DRVNG_DSTNC  1230 non-null   float64
dtypes: float64(1), object(3)
memory usage: 38.6+ KB


In [12]:
# expdf = STD_BD_KOSIS
# table_nm = 'STD_BD_KOSIS'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     # if 'float' in expdf[column].dtype.name:
#     #     sql += column + ' float'
#     # elif 'int' in expdf[column].dtype.name:
#     #     sql += column + ' number'
#     # else:
#     sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 1s
# we.import_from_pandas(expdf, table_nm)

# BD1

## STD_BD_GRD4_CAR_CURSTT

In [7]:
STD_BD_GRD4_CAR_CURSTT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_GRD4_CAR_CURSTT.csv'), low_memory=False)
STD_BD_GRD4_CAR_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1390266 entries, 0 to 1390265
Data columns (total 27 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   IDX                    1390266 non-null  int64  
 1   LOAD_DT                1390266 non-null  int64  
 2   CRTR_YM                1390266 non-null  float64
 3   VIN                    1390266 non-null  object 
 4   MANG_MNG_NO            1390266 non-null  object 
 5   VHCTY_CD               1390264 non-null  object 
 6   PURPS_CD2              1390266 non-null  object 
 7   YRIDNW                 1390266 non-null  int64  
 8   VHCTY_TY               1390266 non-null  object 
 9   FUEL_CD                1390197 non-null  object 
 10  STDG_CD                1390266 non-null  int64  
 11  CTPV_NM                1390266 non-null  object 
 12  SGG_NM                 1382812 non-null  object 
 13  VHCNM                  1390266 non-null  object 
 14  MNFCTR_NM         

In [8]:
expdf = STD_BD_GRD4_CAR_CURSTT
table_nm = 'STD_BD_GRD4_CAR_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 9s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD4_SI

In [9]:
STD_BD_GRD4_SI = pd.read_csv(os.path.join(sys_fold, 'STD_BD_GRD4_SI.csv'))
STD_BD_GRD4_SI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1052697 entries, 0 to 1052696
Data columns (total 11 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   IDX                    1052697 non-null  int64  
 1   LOAD_DT                1052697 non-null  int64  
 2   VIN                    1052697 non-null  object 
 3   MANG_MNG_NO            1052697 non-null  object 
 4   VHCNM                  1052697 non-null  object 
 5   MNFCTR_NM              1052680 non-null  object 
 6   EXHST_GAS_CERT_NO_MOD  1046165 non-null  object 
 7   INSP_MTHD              1047878 non-null  object 
 8   DRVNG_DSTNC            1047672 non-null  float64
 9   VHCAG                  1052697 non-null  int64  
 10  SI                     1006355 non-null  float64
dtypes: float64(2), int64(3), object(6)
memory usage: 88.3+ MB


In [10]:
expdf = STD_BD_GRD4_SI
table_nm = 'STD_BD_GRD4_SI'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 6s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD4_CAR_CURSTT_TOT

In [11]:
STD_BD_GRD4_CAR_CURSTT_TOT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_GRD4_CAR_CURSTT_TOT.csv'))
STD_BD_GRD4_CAR_CURSTT_TOT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1403 entries, 0 to 1402
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   IDX                    1403 non-null   int64  
 1   LOAD_DT                1403 non-null   int64  
 2   VHCNM                  1403 non-null   object 
 3   MNFCTR_NM              1403 non-null   object 
 4   MANG_MNG_NO            1403 non-null   object 
 5   EXHST_GAS_CERT_NO_MOD  1392 non-null   object 
 6   INSP_MTHD              1403 non-null   object 
 7   LOWR_QRT               1403 non-null   float64
 8   MID_QRT                1403 non-null   float64
 9   UP_QRT                 1403 non-null   float64
 10  VHCL_MKCNT             1403 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 120.7+ KB


In [12]:
expdf = STD_BD_GRD4_CAR_CURSTT_TOT
table_nm = 'STD_BD_GRD4_CAR_CURSTT_TOT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_GRD4_CERT_NO_RVW

In [13]:
# 0s
STD_BD_DAT_GRD4_CERT_NO_RVW = pd.read_csv(os.path.join(sys_fold, 'STD_BD_DAT_GRD4_CERT_NO_RVW.csv'))
STD_BD_DAT_GRD4_CERT_NO_RVW.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017 entries, 0 to 1016
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   IDX                1017 non-null   int64  
 1   EXHST_GAS_CERT_NO  1006 non-null   object 
 2   INSP_MTHD          1017 non-null   object 
 3   RVW_SE             1017 non-null   object 
 4   RPRS_MNFCTR_NM     1017 non-null   object 
 5   RPRS_VHCNM         1017 non-null   object 
 6   MANG_MNG_NO        1017 non-null   object 
 7   LOWR_QRT           1017 non-null   float64
 8   MID_QRT            1017 non-null   float64
 9   UP_QRT             1017 non-null   float64
 10  LOAD_DT            1017 non-null   int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 87.5+ KB


In [14]:
expdf = STD_BD_DAT_GRD4_CERT_NO_RVW
table_nm = 'STD_BD_DAT_GRD4_CERT_NO_RVW'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

In [15]:
sql

'create or replace table STD_BD_DAT_GRD4_CERT_NO_RVW( \nIDX number,\nEXHST_GAS_CERT_NO varchar(255),\nINSP_MTHD varchar(255),\nRVW_SE varchar(255),\nRPRS_MNFCTR_NM varchar(255),\nRPRS_VHCNM varchar(255),\nMANG_MNG_NO varchar(255),\nLOWR_QRT float,\nMID_QRT float,\nUP_QRT float,\nLOAD_DT number\n)'

## STD_BD_DAT_GRD4_SI

In [16]:
# 0s
STD_BD_DAT_GRD4_SI = pd.read_csv(os.path.join(sys_fold, 'STD_BD_DAT_GRD4_SI.csv'))
STD_BD_DAT_GRD4_SI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386 entries, 0 to 385
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   IDX                386 non-null    int64  
 1   EXHST_GAS_CERT_NO  382 non-null    object 
 2   INSP_MTHD          386 non-null    object 
 3   RPRS_MNFCTR_NM     386 non-null    object 
 4   RPRS_VHCNM         386 non-null    object 
 5   RPRS_VHCTY_CD      386 non-null    object 
 6   RPRS_FUEL          386 non-null    object 
 7   FRST_YRIDNW        386 non-null    int64  
 8   SI                 378 non-null    float64
 9   LOAD_DT            386 non-null    int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 30.3+ KB


In [17]:
expdf = STD_BD_DAT_GRD4_SI
table_nm = 'STD_BD_DAT_GRD4_SI'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

In [18]:
sql

'create or replace table STD_BD_DAT_GRD4_SI( \nIDX number,\nEXHST_GAS_CERT_NO varchar(255),\nINSP_MTHD varchar(255),\nRPRS_MNFCTR_NM varchar(255),\nRPRS_VHCNM varchar(255),\nRPRS_VHCTY_CD varchar(255),\nRPRS_FUEL varchar(255),\nFRST_YRIDNW number,\nSI float,\nLOAD_DT number\n)'

## STD_BD_GRD4_ELPDSRC_CURSTT

In [19]:
STD_BD_GRD4_ELPDSRC_CURSTT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_GRD4_ELPDSRC_CURSTT.csv'))
STD_BD_GRD4_ELPDSRC_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1411482 entries, 0 to 1411481
Data columns (total 13 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   IDX                  1411482 non-null  int64  
 1   CRTR_YM              1411482 non-null  float64
 2   VIN                  1411482 non-null  object 
 3   STDG_CD              1411482 non-null  int64  
 4   VHCTY_CD             1411482 non-null  object 
 5   PURPS_CD2            1411482 non-null  object 
 6   FUEL_CD              1411482 non-null  object 
 7   VHCTY_TY             1411482 non-null  object 
 8   CTPV                 1411482 non-null  object 
 9   SGG                  1402796 non-null  object 
 10  ELPDSRC_STTS_CD      30783 non-null    object 
 11  ELPDSRC_LST_APRV_YN  30783 non-null    object 
 12  LOAD_DT              1411482 non-null  int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 140.0+ MB


In [20]:
expdf = STD_BD_GRD4_ELPDSRC_CURSTT
table_nm = 'STD_BD_GRD4_ELPDSRC_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD4_MLSFC_RSLT

In [21]:
STD_BD_GRD4_MLSFC_RSLT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_GRD4_MLSFC_RSLT.csv'))
STD_BD_GRD4_MLSFC_RSLT.info()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_20100\3387438605.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  STD_BD_GRD4_MLSFC_RSLT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_GRD4_MLSFC_RSLT.csv'))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1390266 entries, 0 to 1390265
Data columns (total 15 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   IDX           1390266 non-null  int64 
 1   LOAD_DT       1390266 non-null  int64 
 2   VIN           1390266 non-null  object
 3   MANG_MNG_NO   1390266 non-null  object
 4   VHCTY_CD      1390264 non-null  object
 5   PURPS_CD2     1390266 non-null  object
 6   YRIDNW        1390266 non-null  int64 
 7   VHCTY_TY      1390266 non-null  object
 8   FUEL_CD       1390197 non-null  object
 9   STDG_CD       1390266 non-null  int64 
 10  CTPV_NM       1390266 non-null  object
 11  SGG_NM        1382812 non-null  object
 12  DPF_MNTNG_YN  1052613 non-null  object
 13  GRD4_MLSFC    1390266 non-null  object
 14  STDG_CD_MOD   1390266 non-null  int64 
dtypes: int64(5), object(10)
memory usage: 159.1+ MB


In [22]:
expdf = STD_BD_GRD4_MLSFC_RSLT
table_nm = 'STD_BD_GRD4_MLSFC_RSLT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 7s
we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_GRD4_MLSFC

In [23]:
# 0s
STD_BD_DAT_GRD4_MLSFC = pd.read_csv(os.path.join(sys_fold, 'STD_BD_DAT_GRD4_MLSFC.csv'))
STD_BD_DAT_GRD4_MLSFC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8298 entries, 0 to 8297
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   IDX        8298 non-null   int64  
 1   FUEL_CD    8298 non-null   object 
 2   CTPV       8298 non-null   object 
 3   SGG        8298 non-null   object 
 4   VHCTY_CD   8298 non-null   object 
 5   VHCTY_TY   8298 non-null   object 
 6   PURPS_CD2  8298 non-null   object 
 7   A_MKCNT    5066 non-null   float64
 8   B_MKCNT    3788 non-null   float64
 9   C_MKCNT    3332 non-null   float64
 10  D_MKCNT    779 non-null    float64
 11  X_MKCNT    3367 non-null   float64
 12  YR         8298 non-null   int64  
 13  LOAD_DT    8298 non-null   int64  
dtypes: float64(5), int64(3), object(6)
memory usage: 907.7+ KB


In [24]:
expdf = STD_BD_DAT_GRD4_MLSFC
table_nm = 'STD_BD_DAT_GRD4_MLSFC'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_GRD4_DTL_INFO

In [25]:
# 6s
STD_BD_DAT_GRD4_DTL_INFO = pd.read_csv(os.path.join(sys_fold, 'STD_BD_DAT_GRD4_DTL_INFO.csv'))
STD_BD_DAT_GRD4_DTL_INFO.info()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_20100\2969779950.py:2: DtypeWarning: Columns (11,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  STD_BD_DAT_GRD4_DTL_INFO = pd.read_csv(os.path.join(sys_fold, 'STD_BD_DAT_GRD4_DTL_INFO.csv'))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1906799 entries, 0 to 1906798
Data columns (total 19 columns):
 #   Column           Dtype  
---  ------           -----  
 0   IDX              int64  
 1   VHRNO            object 
 2   VIN              object 
 3   GRD4_MLSFC       object 
 4   VHCTY_CD         object 
 5   VHCTY_TY         object 
 6   PURPS_CD2        object 
 7   FUEL_CD          object 
 8   CTPV             object 
 9   SGG              object 
 10  YRIDNW           int64  
 11  DPF_MNTNG_YN     object 
 12  RDCDVC_KND       object 
 13  FRST_REG_YMD     float64
 14  ELPDSRC_APLY_YN  object 
 15  ELPDSRC_STTS_CD  object 
 16  CHNG_DE          float64
 17  VHCL_ERSR_YN     object 
 18  LOAD_DT          int64  
dtypes: float64(2), int64(3), object(14)
memory usage: 276.4+ MB


In [26]:
expdf = STD_BD_DAT_GRD4_DTL_INFO
table_nm = 'STD_BD_DAT_GRD4_DTL_INFO'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 9s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD4_RGN_CURSTT

In [27]:
STD_BD_GRD4_RGN_CURSTT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_GRD4_RGN_CURSTT.csv'))
STD_BD_GRD4_RGN_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3712 entries, 0 to 3711
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   IDX         3712 non-null   int64  
 1   LOAD_DT     3712 non-null   int64  
 2   CRTR_YM     3712 non-null   float64
 3   YR          3712 non-null   int64  
 4   MM          3712 non-null   int64  
 5   FUEL_CD     3712 non-null   object 
 6   CTPV        3712 non-null   object 
 7   SGG         3696 non-null   object 
 8   VHCL_MKCNT  3712 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 261.1+ KB


In [28]:
expdf = STD_BD_GRD4_RGN_CURSTT
table_nm = 'STD_BD_GRD4_RGN_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD4_RGN_CURSTT_MOD

In [29]:
STD_BD_GRD4_RGN_CURSTT_MOD = pd.read_csv(os.path.join(sys_fold, 'STD_BD_GRD4_RGN_CURSTT_MOD.csv'))
STD_BD_GRD4_RGN_CURSTT_MOD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272 entries, 0 to 271
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   IDX         272 non-null    int64  
 1   LOAD_DT     272 non-null    int64  
 2   CRTR_Y      272 non-null    int64  
 3   CTPV        272 non-null    object 
 4   VHCTY_CD    272 non-null    object 
 5   VHCL_MKCNT  272 non-null    float64
dtypes: float64(1), int64(3), object(2)
memory usage: 12.9+ KB


In [30]:
expdf = STD_BD_GRD4_RGN_CURSTT_MOD
table_nm = 'STD_BD_GRD4_RGN_CURSTT_MOD'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_GRD4_CAR_CURSTT

In [31]:
# 1s
STD_BD_DAT_GRD4_CAR_CURSTT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_DAT_GRD4_CAR_CURSTT.csv'))
STD_BD_DAT_GRD4_CAR_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266112 entries, 0 to 266111
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   IDX                  266112 non-null  int64  
 1   YR                   266112 non-null  int64  
 2   MM                   266112 non-null  int64  
 3   CTPV                 266112 non-null  object 
 4   SGG                  266112 non-null  object 
 5   FUEL_CD              266112 non-null  object 
 6   VHCTY_CD             266112 non-null  object 
 7   VHCTY_TY             266112 non-null  object 
 8   PURPS_CD2            266112 non-null  object 
 9   VHCL_MKCNT           266112 non-null  float64
 10  DEC_RT               266112 non-null  float64
 11  RDCDVC_EXTRNS_MKCNT  266112 non-null  float64
 12  RDCDVC_UNAT_MKCNT    266112 non-null  float64
 13  ELPDSRC_MKCNT        266112 non-null  float64
 14  LOAD_DT              266112 non-null  int64  
dtypes: float64(5), in

In [32]:
expdf = STD_BD_DAT_GRD4_CAR_CURSTT
table_nm = 'STD_BD_DAT_GRD4_CAR_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 2s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD4_EXHST_GAS_MSS_TOT

In [33]:
STD_BD_GRD4_EXHST_GAS_MSS_TOT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_GRD4_EXHST_GAS_MSS_TOT.csv'))
STD_BD_GRD4_EXHST_GAS_MSS_TOT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   IDX              231 non-null    int64  
 1   YR               231 non-null    int64  
 2   CTPV             231 non-null    object 
 3   SGG              231 non-null    object 
 4   CO_EXHST_MSS     231 non-null    float64
 5   HC_EXHST_MSS     231 non-null    float64
 6   NOX_EXHST_MSS    231 non-null    float64
 7   PM10_EXHST_MSS   231 non-null    float64
 8   PM2_5_EXHST_MSS  231 non-null    float64
 9   LOAD_DT          231 non-null    int64  
 10  CRTR_YM          231 non-null    float64
dtypes: float64(6), int64(3), object(2)
memory usage: 20.0+ KB


In [34]:
expdf = STD_BD_GRD4_EXHST_GAS_MSS_TOT
table_nm = 'STD_BD_GRD4_EXHST_GAS_MSS_TOT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 7s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD4_EXHST_GAS_MSS

In [35]:
STD_BD_GRD4_EXHST_GAS_MSS = pd.read_csv(os.path.join(sys_fold, 'STD_BD_GRD4_EXHST_GAS_MSS.csv'))
STD_BD_GRD4_EXHST_GAS_MSS.info()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_20100\2736260677.py:1: DtypeWarning: Columns (5,30) have mixed types. Specify dtype option on import or set low_memory=False.
  STD_BD_GRD4_EXHST_GAS_MSS = pd.read_csv(os.path.join(sys_fold, 'STD_BD_GRD4_EXHST_GAS_MSS.csv'))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1390161 entries, 0 to 1390160
Data columns (total 37 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   IDX                     1390161 non-null  int64  
 1   LOAD_DT                 1390161 non-null  int64  
 2   CRTR_YM                 1390161 non-null  float64
 3   VIN                     1390161 non-null  object 
 4   VHRNO                   1390161 non-null  object 
 5   MANG_MNG_NO             1390161 non-null  object 
 6   VHCTY_CD                1390159 non-null  object 
 7   PURPS_CD2               1390161 non-null  object 
 8   FRST_REG_YMD            1390159 non-null  object 
 9   YRIDNW                  1390161 non-null  int64  
 10  FBCTN_YMD               1226003 non-null  float64
 11  EXHST_GAS_INSP_VLD_YMD  1390156 non-null  float64
 12  VHCNM                   1390161 non-null  object 
 13  VHCTY_CL_CD             1390161 non-null  object 
 14  VH

In [36]:
expdf = STD_BD_GRD4_EXHST_GAS_MSS
table_nm = 'STD_BD_GRD4_EXHST_GAS_MSS'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 30.7s
we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_GRD4_EXHST_MSS_CURSTT

In [37]:
# 0s
STD_BD_DAT_GRD4_EXHST_MSS_CURSTT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_DAT_GRD4_EXHST_MSS_CURSTT.csv'))
STD_BD_DAT_GRD4_EXHST_MSS_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8273 entries, 0 to 8272
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   IDX                 8273 non-null   int64  
 1   YR                  8273 non-null   int64  
 2   CTPV                8273 non-null   object 
 3   SGG                 8273 non-null   object 
 4   FUEL_CD             8273 non-null   object 
 5   VHCTY_CD            8273 non-null   object 
 6   VHCTY_TY            8273 non-null   object 
 7   PURPS_CD2           8273 non-null   object 
 8   CO_EXHST_MSS_SM     8273 non-null   float64
 9   HC_EXHST_MSS_SM     8273 non-null   float64
 10  NOX_EXHST_MSS_SM    8273 non-null   float64
 11  PM10_EXHST_MSS_SM   8273 non-null   float64
 12  PM2_5_EXHST_MSS_SM  8273 non-null   float64
 13  LOAD_DT             8273 non-null   int64  
dtypes: float64(5), int64(3), object(6)
memory usage: 905.0+ KB


In [38]:
expdf = STD_BD_DAT_GRD4_EXHST_MSS_CURSTT
table_nm = 'STD_BD_DAT_GRD4_EXHST_MSS_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

In [39]:
sql

'create or replace table STD_BD_DAT_GRD4_EXHST_MSS_CURSTT( \nIDX number,\nYR number,\nCTPV varchar(255),\nSGG varchar(255),\nFUEL_CD varchar(255),\nVHCTY_CD varchar(255),\nVHCTY_TY varchar(255),\nPURPS_CD2 varchar(255),\nCO_EXHST_MSS_SM float,\nHC_EXHST_MSS_SM float,\nNOX_EXHST_MSS_SM float,\nPM10_EXHST_MSS_SM float,\nPM2_5_EXHST_MSS_SM float,\nLOAD_DT number\n)'

## STD_BD_DAT_GRD4_MEVLU

In [40]:
# 0s
STD_BD_DAT_GRD4_MEVLU = pd.read_csv(os.path.join(sys_fold, 'STD_BD_DAT_GRD4_MEVLU.csv'))
STD_BD_DAT_GRD4_MEVLU.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8273 entries, 0 to 8272
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   IDX                   8273 non-null   int64  
 1   YR                    8273 non-null   int64  
 2   CTPV                  8273 non-null   object 
 3   SGG                   8273 non-null   object 
 4   FUEL_CD               8273 non-null   object 
 5   VHCTY_CD              8273 non-null   object 
 6   VHCTY_TY              8273 non-null   object 
 7   PURPS_CD2             8273 non-null   object 
 8   CO_EXHST_MSS_AVRG     8273 non-null   float64
 9   HC_EXHST_MSS_AVRG     8273 non-null   float64
 10  NOX_EXHST_MSS_AVRG    8273 non-null   float64
 11  PM10_EXHST_MSS_AVRG   8273 non-null   float64
 12  PM2_5_EXHST_MSS_AVRG  8273 non-null   float64
 13  LOAD_DT               8273 non-null   int64  
dtypes: float64(5), int64(3), object(6)
memory usage: 905.0+ KB


In [41]:
expdf = STD_BD_DAT_GRD4_MEVLU
table_nm = 'STD_BD_DAT_GRD4_MEVLU'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

# BD3

## STD_BD_CAR_CURSTT_MOD

In [42]:
STD_BD_CAR_CURSTT_MOD = pd.read_csv(os.path.join(sys_fold, 'STD_BD_CAR_CURSTT_MOD.csv'))
STD_BD_CAR_CURSTT_MOD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408 entries, 0 to 407
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   IDX               408 non-null    int64  
 1   YR                408 non-null    int64  
 2   RGN               408 non-null    object 
 3   CTPV_NM           408 non-null    object 
 4   EXHST_GAS_GRD_CD  408 non-null    object 
 5   VHCL_MKCNT        408 non-null    float64
 6   CRTR_YM           408 non-null    int64  
 7   LOAD_DT           408 non-null    int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 25.6+ KB


In [43]:
expdf = STD_BD_CAR_CURSTT_MOD
table_nm = 'STD_BD_CAR_CURSTT_MOD'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## STD_BD_CAR_CURSTT_MOD2

In [44]:
STD_BD_CAR_CURSTT_MOD2 = pd.read_csv(os.path.join(sys_fold, 'STD_BD_CAR_CURSTT_MOD2.csv'))
STD_BD_CAR_CURSTT_MOD2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   IDX               120 non-null    int64  
 1   YR                120 non-null    int64  
 2   FUEL_CD           120 non-null    object 
 3   EXHST_GAS_GRD_CD  120 non-null    object 
 4   VHCL_MKCNT        120 non-null    float64
 5   CRTR_YM           120 non-null    int64  
 6   LOAD_DT           120 non-null    int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 6.7+ KB


In [45]:
expdf = STD_BD_CAR_CURSTT_MOD2
table_nm = 'STD_BD_CAR_CURSTT_MOD2'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## STD_BD_CAR_CURSTT

In [46]:
STD_BD_CAR_CURSTT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_CAR_CURSTT.csv'))
STD_BD_CAR_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25615830 entries, 0 to 25615829
Data columns (total 13 columns):
 #   Column             Dtype  
---  ------             -----  
 0   IDX                int64  
 1   LOAD_DT            int64  
 2   BSPL_STDG_CD       int64  
 3   VHCTY_CD           object 
 4   PURPS_CD2          object 
 5   VIN                object 
 6   MANG_MNG_NO        object 
 7   EXHST_GAS_CERT_NO  object 
 8   EXHST_GAS_GRD_CD   float64
 9   VHCL_ERSR_YN       object 
 10  FUEL_CD            object 
 11  CTPV               object 
 12  SGG                object 
dtypes: float64(1), int64(3), object(9)
memory usage: 2.5+ GB


In [47]:
expdf = STD_BD_CAR_CURSTT
table_nm = 'STD_BD_CAR_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## STD_BD_CAR_REG_MKCNT

In [48]:
STD_BD_CAR_REG_MKCNT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_CAR_REG_MKCNT.csv'))
STD_BD_CAR_REG_MKCNT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   IDX         340 non-null    int64  
 1   LOAD_DT     340 non-null    int64  
 2   YR          340 non-null    int64  
 3   FUEL_CD     340 non-null    object 
 4   CTPV        340 non-null    object 
 5   VHCL_MKCNT  340 non-null    float64
 6   FUEL_RT     340 non-null    float64
dtypes: float64(2), int64(3), object(2)
memory usage: 18.7+ KB


In [49]:
expdf = STD_BD_CAR_REG_MKCNT
table_nm = 'STD_BD_CAR_REG_MKCNT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## STD_BD_CAR_PRET

In [50]:
STD_BD_CAR_PRET = pd.read_csv(os.path.join(sys_fold, 'STD_BD_CAR_PRET.csv'))
STD_BD_CAR_PRET.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   IDX             17 non-null     int64  
 1   LOAD_DT         17 non-null     int64  
 2   YR              17 non-null     int64  
 3   DSL             4 non-null      float64
 4   GSL             4 non-null      float64
 5   LPG             4 non-null      float64
 6   DSL_PRET        13 non-null     float64
 7   DSL_PRET_BSPLN  13 non-null     float64
 8   DSL_PRET_AKM    13 non-null     float64
 9   GSL_PRET        13 non-null     float64
 10  GSL_PRET_BSPLN  13 non-null     float64
 11  GSL_PRET_AKM    13 non-null     float64
 12  LPG_PRET        13 non-null     float64
 13  LPG_PRET_BSPLN  13 non-null     float64
 14  LPG_PRET_AKM    13 non-null     float64
dtypes: float64(12), int64(3)
memory usage: 2.1 KB


In [51]:
expdf = STD_BD_CAR_PRET
table_nm = 'STD_BD_CAR_PRET'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## STD_BD_HYBRD_CAR_PRET

In [52]:
STD_BD_HYBRD_CAR_PRET = pd.read_csv(os.path.join(sys_fold, 'STD_BD_HYBRD_CAR_PRET.csv'))
STD_BD_HYBRD_CAR_PRET.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   IDX              17 non-null     int64  
 1   LOAD_DT          17 non-null     int64  
 2   YR               17 non-null     int64  
 3   GSLH             4 non-null      float64
 4   GSLH_PRET        13 non-null     float64
 5   GSLH_PRET_BSPLN  13 non-null     float64
 6   GSLH_PRET_AKM    13 non-null     float64
 7   DSLH             4 non-null      float64
 8   DSLH_PRET        13 non-null     float64
 9   DSLH_PRET_BSPLN  13 non-null     float64
 10  DSLH_PRET_AKM    13 non-null     float64
 11  LPGH             4 non-null      float64
 12  LPGH_PRET        13 non-null     float64
 13  LPGH_PRET_BSPLN  13 non-null     float64
 14  LPGH_PRET_AKM    13 non-null     float64
dtypes: float64(12), int64(3)
memory usage: 2.1 KB


In [53]:
expdf = STD_BD_HYBRD_CAR_PRET
table_nm = 'STD_BD_HYBRD_CAR_PRET'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 7s
we.import_from_pandas(expdf, table_nm)

## STD_BD_FUEL_GRD_VHCL_CURSTT_PRET

In [54]:
STD_BD_FUEL_GRD_VHCL_CURSTT_PRET = pd.read_csv(os.path.join(sys_fold, 'STD_BD_FUEL_GRD_VHCL_CURSTT_PRET.csv'))
STD_BD_FUEL_GRD_VHCL_CURSTT_PRET.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   IDX               255 non-null    int64  
 1   YR                255 non-null    int64  
 2   FUEL_CD           255 non-null    object 
 3   EXHST_GAS_GRD_CD  255 non-null    int64  
 4   VHCL_MKCNT        60 non-null     float64
 5   VHCL_PRET         195 non-null    float64
 6   LOAD_DT           255 non-null    int64  
dtypes: float64(2), int64(4), object(1)
memory usage: 14.1+ KB


In [55]:
expdf = STD_BD_FUEL_GRD_VHCL_CURSTT_PRET
table_nm = 'STD_BD_FUEL_GRD_VHCL_CURSTT_PRET'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## STD_BD_ECO_CAR_PRET

In [56]:
STD_BD_ECO_CAR_PRET = pd.read_csv(os.path.join(sys_fold, 'STD_BD_ECO_CAR_PRET.csv'))
STD_BD_ECO_CAR_PRET.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   IDX               17 non-null     int64  
 1   LOAD_DT           17 non-null     int64  
 2   YR                17 non-null     int64  
 3   BTYCR             4 non-null      float64
 4   BTYCR_PRET        13 non-null     float64
 5   BTYCR_PRET_BSPLN  13 non-null     float64
 6   BTYCR_PRET_AKM    13 non-null     float64
 7   HY                4 non-null      float64
 8   HY_PRET           13 non-null     float64
 9   HY_PRET_BSPLN     13 non-null     float64
 10  HY_PRET_AKM       13 non-null     float64
dtypes: float64(8), int64(3)
memory usage: 1.6 KB


In [57]:
expdf = STD_BD_ECO_CAR_PRET
table_nm = 'STD_BD_ECO_CAR_PRET'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 7s
we.import_from_pandas(expdf, table_nm)

## STD_BD_ERSR_RSLT

In [58]:
STD_BD_ERSR_RSLT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_ERSR_RSLT.csv'))
STD_BD_ERSR_RSLT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4792996 entries, 0 to 4792995
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   IDX               int64  
 1   LOAD_DT           int64  
 2   BSPL_STDG_CD      int64  
 3   CTPV              object 
 4   SGG               object 
 5   VIN               object 
 6   CHG_YMD           int64  
 7   EXHST_GAS_GRD_CD  float64
 8   FUEL_CD           object 
dtypes: float64(1), int64(4), object(4)
memory usage: 329.1+ MB


In [59]:
expdf = STD_BD_ERSR_RSLT
table_nm = 'STD_BD_ERSR_RSLT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 12.6s
we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_GRD_CURSTT

In [60]:
# 1s
STD_BD_DAT_GRD_CURSTT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_DAT_GRD_CURSTT.csv'))
STD_BD_DAT_GRD_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212160 entries, 0 to 212159
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   IDX               212160 non-null  int64  
 1   CTPV              212160 non-null  object 
 2   YR                212160 non-null  int64  
 3   MM                212160 non-null  int64  
 4   EXHST_GAS_GRD_CD  212160 non-null  int64  
 5   FUEL_CD           212160 non-null  object 
 6   VHCTY_CD          212160 non-null  object 
 7   VHCTY_TY          212160 non-null  object 
 8   PURPS_CD2         212160 non-null  object 
 9   VHCL_MKCNT        212160 non-null  float64
 10  ERSR_MKCNT        212160 non-null  float64
 11  YR_FUEL_VHCL_RT   212160 non-null  float64
 12  LOAD_DT           212160 non-null  int64  
dtypes: float64(3), int64(5), object(5)
memory usage: 21.0+ MB


In [61]:
expdf = STD_BD_DAT_GRD_CURSTT
table_nm = 'STD_BD_DAT_GRD_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 12.6s
we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_FUEL_CAR_DEC

In [62]:
# 0s
STD_BD_DAT_FUEL_CAR_DEC = pd.read_csv(os.path.join(sys_fold, 'STD_BD_DAT_FUEL_CAR_DEC.csv'))
STD_BD_DAT_FUEL_CAR_DEC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224 entries, 0 to 1223
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   IDX               1224 non-null   int64  
 1   YR                1224 non-null   int64  
 2   QRT               1224 non-null   int64  
 3   EXHST_GAS_GRD_CD  1224 non-null   object 
 4   FUEL_CD           1224 non-null   object 
 5   LPG_MKCNT         96 non-null     float64
 6   LPG_PRET          312 non-null    float64
 7   LPG_PRET_BSPLN    312 non-null    float64
 8   LPG_PRET_AKM      312 non-null    float64
 9   DSL_MKCNT         96 non-null     float64
 10  DSL_PRET          312 non-null    float64
 11  DSL_PRET_BSPLN    312 non-null    float64
 12  DSL_PRET_AKM      312 non-null    float64
 13  GSL_MKCNT         96 non-null     float64
 14  GSL_PRET          312 non-null    float64
 15  GSL_PRET_BSPLN    312 non-null    float64
 16  GSL_PRET_AKM      312 non-null    float64


In [63]:
expdf = STD_BD_DAT_FUEL_CAR_DEC
table_nm = 'STD_BD_DAT_FUEL_CAR_DEC'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 12.6s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD5_ELPDSRC_CURSTT

In [64]:
STD_BD_GRD5_ELPDSRC_CURSTT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_GRD5_ELPDSRC_CURSTT.csv'))
STD_BD_GRD5_ELPDSRC_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   IDX         73 non-null     int64  
 1   RGN         68 non-null     object 
 2   YR          68 non-null     float64
 3   VHCL_MKCNT  68 non-null     object 
 4   VHCL_REDE   68 non-null     object 
 5   ELPDSRC     68 non-null     object 
 6   NTRL_DCLN   68 non-null     object 
 7   LOAD_DT     73 non-null     int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 4.7+ KB


In [65]:
expdf = STD_BD_GRD5_ELPDSRC_CURSTT
table_nm = 'STD_BD_GRD5_ELPDSRC_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

In [66]:
sql

'create or replace table STD_BD_GRD5_ELPDSRC_CURSTT( \nIDX number,\nRGN varchar(255),\nYR float,\nVHCL_MKCNT varchar(255),\nVHCL_REDE varchar(255),\nELPDSRC varchar(255),\nNTRL_DCLN varchar(255),\nLOAD_DT number\n)'

## STD_BD_GRD5_LEM_N_MOD

In [67]:
STD_BD_GRD5_LEM_N_MOD = pd.read_csv(os.path.join(sys_fold, 'STD_BD_GRD5_LEM_N_MOD.csv'))
STD_BD_GRD5_LEM_N_MOD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   IDX         12 non-null     int64 
 1   RGN         12 non-null     object
 2   SEASON      12 non-null     object
 3   VHCL_MKCNT  12 non-null     object
 4   LOAD_DT     12 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 608.0+ bytes


In [68]:
expdf = STD_BD_GRD5_LEM_N_MOD
table_nm = 'STD_BD_GRD5_LEM_N_MOD'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

In [69]:
sql

'create or replace table STD_BD_GRD5_LEM_N_MOD( \nIDX number,\nRGN varchar(255),\nSEASON varchar(255),\nVHCL_MKCNT varchar(255),\nLOAD_DT number\n)'

## STD_BD_SEASON_CRDN_NOCS_CURSTT

In [70]:
STD_BD_SEASON_CRDN_NOCS_CURSTT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_SEASON_CRDN_NOCS_CURSTT.csv'))
STD_BD_SEASON_CRDN_NOCS_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 973974 entries, 0 to 973973
Data columns (total 12 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   IDX                    973974 non-null  int64  
 1   LOAD_DT                973974 non-null  int64  
 2   VIN                    973974 non-null  object 
 3   RGN                    973974 non-null  object 
 4   CTPV                   973974 non-null  object 
 5   DPF_EXTRNS_YN          973974 non-null  object 
 6   VHCTY_CD               973962 non-null  object 
 7   VHCTY_TY               973974 non-null  object 
 8   SEASON_1ODR_CRDN_NOCS  973974 non-null  float64
 9   SEASON_2ODR_CRDN_NOCS  973974 non-null  float64
 10  SEASON_3ODR_CRDN_NOCS  973974 non-null  float64
 11  SEASON_4ODR_CRDN_NOCS  973974 non-null  float64
dtypes: float64(4), int64(2), object(6)
memory usage: 89.2+ MB


In [71]:
expdf = STD_BD_SEASON_CRDN_NOCS_CURSTT
table_nm = 'STD_BD_SEASON_CRDN_NOCS_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 12.6s
we.import_from_pandas(expdf, table_nm)

## STD_BD_SEASON_DY_AVRG_CRDN_NOCS

In [72]:
STD_BD_SEASON_DY_AVRG_CRDN_NOCS = pd.read_csv(os.path.join(sys_fold, 'STD_BD_SEASON_DY_AVRG_CRDN_NOCS.csv'))
STD_BD_SEASON_DY_AVRG_CRDN_NOCS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   IDX                4 non-null      int64  
 1   SEASON_ORD         4 non-null      object 
 2   DY_AVRG_CRDN_NOCS  4 non-null      float64
 3   LOAD_DT            4 non-null      int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 256.0+ bytes


In [73]:
expdf = STD_BD_SEASON_DY_AVRG_CRDN_NOCS
table_nm = 'STD_BD_SEASON_DY_AVRG_CRDN_NOCS'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

In [74]:
sql

'create or replace table STD_BD_SEASON_DY_AVRG_CRDN_NOCS( \nIDX number,\nSEASON_ORD varchar(255),\nDY_AVRG_CRDN_NOCS float,\nLOAD_DT number\n)'

## STD_BD_ORDITM_DSCL_CURSTT

In [75]:
STD_BD_ORDITM_DSCL_CURSTT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_ORDITM_DSCL_CURSTT.csv'))
STD_BD_ORDITM_DSCL_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84293 entries, 0 to 84292
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   IDX        84293 non-null  int64  
 1   VIN        84293 non-null  object 
 2   DSCL_YR    84293 non-null  int64  
 3   DSCL_NOCS  84293 non-null  float64
 4   RGN        84293 non-null  object 
 5   CTPV       84179 non-null  object 
 6   LOAD_DT    84293 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 4.5+ MB


In [76]:
expdf = STD_BD_ORDITM_DSCL_CURSTT
table_nm = 'STD_BD_ORDITM_DSCL_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

## STD_BD_SEASON_DSCL_RGN_CURSTT

In [77]:
STD_BD_SEASON_DSCL_RGN_CURSTT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_SEASON_DSCL_RGN_CURSTT.csv'))
STD_BD_SEASON_DSCL_RGN_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264089 entries, 0 to 1264088
Data columns (total 11 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   IDX                    1264089 non-null  int64  
 1   DSCL_RGN               1264089 non-null  object 
 2   DSCL_CTPV              1264089 non-null  object 
 3   VIN                    1264089 non-null  object 
 4   SEASON_1ODR_CRDN_NOCS  0 non-null        float64
 5   SEASON_2ODR_CRDN_NOCS  146276 non-null   float64
 6   SEASON_3ODR_CRDN_NOCS  75964 non-null    float64
 7   SEASON_4ODR_CRDN_NOCS  49717 non-null    float64
 8   VHCTY_CD               840194 non-null   object 
 9   VHCTY_TY               840194 non-null   object 
 10  LOAD_DT                1264089 non-null  int64  
dtypes: float64(4), int64(2), object(5)
memory usage: 106.1+ MB


In [78]:
expdf = STD_BD_SEASON_DSCL_RGN_CURSTT
table_nm = 'STD_BD_SEASON_DSCL_RGN_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 12.6s
we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_GRD5_REDUC_BIZ

In [79]:
# 0s
STD_BD_DAT_GRD5_REDUC_BIZ = pd.read_csv(os.path.join(sys_fold, 'STD_BD_DAT_GRD5_REDUC_BIZ.csv'))
STD_BD_DAT_GRD5_REDUC_BIZ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   IDX              4 non-null      int64  
 1   YR               4 non-null      int64  
 2   VHCL_MKCNT       4 non-null      float64
 3   NTRL_DCLN        3 non-null      float64
 4   ELPDSRC          4 non-null      int64  
 5   RDCDVC_1KND      4 non-null      float64
 6   RDCDVC_1KND_SCR  4 non-null      float64
 7   UNMNTNG          4 non-null      float64
 8   LOAD_DT          4 non-null      int64  
dtypes: float64(5), int64(4)
memory usage: 416.0 bytes


In [80]:
expdf = STD_BD_DAT_GRD5_REDUC_BIZ
table_nm = 'STD_BD_DAT_GRD5_REDUC_BIZ'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 12.6s
we.import_from_pandas(expdf, table_nm)

In [81]:
sql

'create or replace table STD_BD_DAT_GRD5_REDUC_BIZ( \nIDX number,\nYR number,\nVHCL_MKCNT float,\nNTRL_DCLN float,\nELPDSRC number,\nRDCDVC_1KND float,\nRDCDVC_1KND_SCR float,\nUNMNTNG float,\nLOAD_DT number\n)'

## STD_BD_DAT_RUN_LMT_CURSTT

In [82]:
# 0s
STD_BD_DAT_RUN_LMT_CURSTT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_DAT_RUN_LMT_CURSTT.csv'))
STD_BD_DAT_RUN_LMT_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   IDX              502 non-null    int64  
 1   SEASON           502 non-null    object 
 2   RGN              502 non-null    object 
 3   CTPV             502 non-null    object 
 4   VHCTY_CD         502 non-null    object 
 5   VHCTY_TY         502 non-null    object 
 6   DSCL_VHCL_MKCNT  502 non-null    int64  
 7   UNLEM            502 non-null    int64  
 8   DUP_DSCL_MKCNT   433 non-null    float64
 9   LOAD_DT          502 non-null    int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 39.3+ KB


In [83]:
expdf = STD_BD_DAT_RUN_LMT_CURSTT
table_nm = 'STD_BD_DAT_RUN_LMT_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 12.6s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD4_DS_CRRLTN_CFFCNT

In [84]:
STD_BD_GRD4_DS_CRRLTN_CFFCNT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_GRD4_DS_CRRLTN_CFFCNT.csv'))
STD_BD_GRD4_DS_CRRLTN_CFFCNT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   IDX                  4 non-null      int64  
 1   LIST                 4 non-null      object 
 2   NOLOD_SMO_MEVLU1     4 non-null      float64
 3   DY_AVRG_DRVNG_DSTNC  4 non-null      float64
 4   RCNT_INSP_ELPSD_WHL  4 non-null      float64
 5   RUN_LMT_NOCS         4 non-null      float64
 6   TOT_CRRLTN_CFFCNT    4 non-null      float64
 7   LOAD_DT              4 non-null      int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 384.0+ bytes


In [85]:
expdf = STD_BD_GRD4_DS_CRRLTN_CFFCNT
table_nm = 'STD_BD_GRD4_DS_CRRLTN_CFFCNT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD4_LEM_PRIO_ORD_SELCT_CURSTT

In [86]:
STD_BD_GRD4_LEM_PRIO_ORD_SELCT_CURSTT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_GRD4_LEM_PRIO_ORD_SELCT_CURSTT.csv'))
STD_BD_GRD4_LEM_PRIO_ORD_SELCT_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1052791 entries, 0 to 1052790
Data columns (total 17 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   IDX                  1052791 non-null  int64  
 1   LOAD_DT              1052791 non-null  int64  
 2   VIN                  1052791 non-null  object 
 3   VHRNO                1052791 non-null  object 
 4   STDG_CD              1052791 non-null  int64  
 5   CTPV_NM              1052791 non-null  object 
 6   SGG_NM               1046559 non-null  object 
 7   YRIDNW               1052791 non-null  int64  
 8   PURPS_CD2            1052791 non-null  object 
 9   VHCTY_CD             1052791 non-null  object 
 10  VHCTY_TY             1052791 non-null  object 
 11  PRIO_GRD             1052791 non-null  int64  
 12  SELCT_PNT            1024883 non-null  float64
 13  NOLOD_SMO_MEVLU1     1047848 non-null  float64
 14  DY_AVRG_DRVNG_DSTNC  1052791 non-null  float64
 15

In [87]:
expdf = STD_BD_GRD4_LEM_PRIO_ORD_SELCT_CURSTT
table_nm = 'STD_BD_GRD4_LEM_PRIO_ORD_SELCT_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 12.6s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD5_DS_CRRLTN_CFFCNT

In [88]:
STD_BD_GRD5_DS_CRRLTN_CFFCNT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_GRD5_DS_CRRLTN_CFFCNT.csv'))
STD_BD_GRD5_DS_CRRLTN_CFFCNT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   IDX                  4 non-null      int64  
 1   LIST                 4 non-null      object 
 2   NOLOD_SMO_MEVLU1     4 non-null      float64
 3   DY_AVRG_DRVNG_DSTNC  4 non-null      float64
 4   RCNT_INSP_ELPSD_WHL  4 non-null      float64
 5   RUN_LMT_NOCS         4 non-null      float64
 6   TOT_CRRLTN_CFFCNT    4 non-null      float64
 7   LOAD_DT              4 non-null      int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 384.0+ bytes


In [89]:
expdf = STD_BD_GRD5_DS_CRRLTN_CFFCNT
table_nm = 'STD_BD_GRD5_DS_CRRLTN_CFFCNT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

## STD_BD_GRD5_LEM_PRIO_ORD_SELCT_CURSTT

In [90]:
STD_BD_GRD5_LEM_PRIO_ORD_SELCT_CURSTT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_GRD5_LEM_PRIO_ORD_SELCT_CURSTT.csv'))
STD_BD_GRD5_LEM_PRIO_ORD_SELCT_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943881 entries, 0 to 943880
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   IDX                  943881 non-null  int64  
 1   VIN                  943881 non-null  object 
 2   VHRNO                943881 non-null  object 
 3   STDG_CD              943881 non-null  int64  
 4   CTPV_NM              943881 non-null  object 
 5   SGG_NM               940392 non-null  object 
 6   YRIDNW               943881 non-null  int64  
 7   PURPS_CD2            943881 non-null  object 
 8   VHCTY_CD             943871 non-null  object 
 9   VHCTY_TY             943881 non-null  object 
 10  RDCDVC               281483 non-null  object 
 11  PRIO_GRD             943881 non-null  int64  
 12  SELCT_PNT            555873 non-null  float64
 13  NOLOD_SMO_MEVLU1     704145 non-null  float64
 14  DY_AVRG_DRVNG_DSTNC  943881 non-null  float64
 15  RCNT_INSP_ELPSD_W

In [91]:
expdf = STD_BD_GRD5_LEM_PRIO_ORD_SELCT_CURSTT
table_nm = 'STD_BD_GRD5_LEM_PRIO_ORD_SELCT_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 12.6s
we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_LEM_SELCT

In [92]:
# 0s
STD_BD_DAT_LEM_SELCT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_DAT_LEM_SELCT.csv'))
STD_BD_DAT_LEM_SELCT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   IDX                  397 non-null    int64  
 1   CTPV                 397 non-null    object 
 2   EXHST_GAS_GRD_CD     397 non-null    int64  
 3   VHCTY_CD             397 non-null    object 
 4   VHCTY_TY             397 non-null    object 
 5   SELCT_MKCNT          397 non-null    float64
 6   SENO1_MKCNT          397 non-null    float64
 7   SENO1_RT             397 non-null    float64
 8   SENO2_MKCNT          397 non-null    float64
 9   SENO2_RT             397 non-null    float64
 10  SENO3_MKCNT          397 non-null    float64
 11  SENO3_RT             397 non-null    float64
 12  RDCDVC_EXTRNS_MKCNT  397 non-null    int64  
 13  LOAD_DT              397 non-null    int64  
dtypes: float64(7), int64(4), object(3)
memory usage: 43.5+ KB


In [93]:
expdf = STD_BD_DAT_LEM_SELCT
table_nm = 'STD_BD_DAT_LEM_SELCT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

## STD_BD_DAT_SELCT_PNT_CURSTT

In [94]:
# 0s
STD_BD_DAT_SELCT_PNT_CURSTT = pd.read_csv(os.path.join(sys_fold, 'STD_BD_DAT_SELCT_PNT_CURSTT.csv'))
STD_BD_DAT_SELCT_PNT_CURSTT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   IDX                  160 non-null    int64  
 1   EXHST_GAS_GRD_CD     160 non-null    int64  
 2   SELCT_PNT_RNG        160 non-null    object 
 3   VHCTY_CD             160 non-null    object 
 4   VHCTY_TY             160 non-null    object 
 5   VHCL_MKCNT           160 non-null    int64  
 6   SMO_MSRMT_VAL        160 non-null    float64
 7   DY_AVRG_DRVNG_DSTNC  160 non-null    float64
 8   RCNT_INSP_ELPSD_WHL  160 non-null    float64
 9   RUN_LMT_NOCS         160 non-null    float64
 10  LOAD_DT              160 non-null    int64  
dtypes: float64(4), int64(4), object(3)
memory usage: 13.9+ KB


In [95]:
expdf = STD_BD_DAT_SELCT_PNT_CURSTT
table_nm = 'STD_BD_DAT_SELCT_PNT_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    # if 'float' in expdf[column].dtype.name:
    #     sql += column + ' float'
    # elif 'int' in expdf[column].dtype.name:
    #     sql += column + ' number'
    # else:
    sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 1s
we.import_from_pandas(expdf, table_nm)

# 동양 요청 파일

## STD_BD_CAR_CURSTT_MOD_4RD_INS

In [96]:
# # 0s
# STD_BD_CAR_CURSTT_MOD_4RD_INS = pd.read_csv(os.path.join(sys_fold, 'STD_BD_CAR_CURSTT_MOD_4RD_INS.csv'))
# STD_BD_CAR_CURSTT_MOD_4RD_INS.info()

In [97]:
# expdf = STD_BD_CAR_CURSTT_MOD_4RD_INS
# table_nm = 'STD_BD_CAR_CURSTT_MOD_4RD_INS'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 1s
# we.import_from_pandas(expdf, table_nm)

In [98]:
# sql

## STD_BD_CAR_CURSTT_MOD2_4RD_INS

In [99]:
# # 0s
# STD_BD_CAR_CURSTT_MOD2_4RD_INS = pd.read_csv(os.path.join(sys_fold, 'STD_BD_CAR_CURSTT_MOD2_4RD_INS.csv'))
# STD_BD_CAR_CURSTT_MOD2_4RD_INS.info()

In [100]:
# expdf = STD_BD_CAR_CURSTT_MOD2_4RD_INS
# table_nm = 'STD_BD_CAR_CURSTT_MOD2_4RD_INS'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 1s
# we.import_from_pandas(expdf, table_nm)

In [101]:
# sql

# code end